In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

from sklearn import base

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

from scipy.stats import chi2_contingency
from sklearn.feature_selection import mutual_info_classif

from sklearn.preprocessing import StandardScaler, RobustScaler

from sklearn.impute import KNNImputer

from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
import lightgbm as lgb
import itertools
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier


from sklearn.feature_selection import RFE
from sklearn.metrics import precision_recall_curve
from collections import defaultdict 

from sklearn.metrics import f1_score, classification_report

import warnings
warnings.filterwarnings('ignore')

import scipy.stats as stats


from sklearn.model_selection import TimeSeriesSplit
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.svm import SVC
from utils import *
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight

from collections import Counter
import inspect
from collections import defaultdict

from sklearn.multiclass import OneVsRestClassifier
# imports dataset

df = pd.read_csv("train_new_feats.csv")

#agreement_reached_predicted = pd.read_csv("agreement_predictions.csv")

#df = pd.concat([df, agreement_reached_predicted], axis=1)

df.columns

#df.set_index("Claim Identifier", inplace=True)

# ------------ Subsets of features  

df.columns

target = [[f"target_{i}" for i in range(1, 9)] + ["Claim Injury Type"] + ["WCB Decision"] + ["Agreement Reached"] + ["Claim Injury Type_encoded"]]
target = [item for sublist in target for item in sublist]
target

binary_target = [f"target_{i}" for i in range(1, 9)]

original_target  = [col for col in target if col not in binary_target]

ordinal_target = ["Claim Injury Type_encoded"]

features = [feat for feat in df.columns if feat not in target]

features = [feat for feat in features if df[feat].dtype != "datetime64[ns]"]

num_feats = [feat for feat in features if df[feat].dtype != "object"]

cat_feats = [feat for feat in features if df[feat].dtype == "object"]
cat_feats_index = [features.index(feat) for feat in cat_feats]


df[features+binary_target].columns

def num_imputing(X_train, X_val):
    feats_imput_max = ["C2_Accident_gap_weeks", "C3_Accident_gap_weeks", "Accident Date_assembly_gap_days", "Hearing_C3 gap_months", "Hearing_C2 gap_months", "Hearing_assembly_gap_months", "Days to First Hearing"]

    feat_imput_min = ["C3-C2_gap_days"]
    
    for feat in X_train.columns:
        if X_train[feat].isna().sum() > 0 or X_val[feat].isna().sum() > 0:
            if feat in feats_imput_max:
                X_train[feat] = X_train[feat].fillna(X_train[feat].max())
                X_val[feat] = X_val[feat].fillna(X_train[feat].max())
            elif feat in feat_imput_min:
                X_train[feat] = X_train[feat].fillna(X_train[feat].min())
                X_val[feat] = X_val[feat].fillna(X_train[feat].min())
            else:
                X_train[feat] = X_train[feat].fillna(X_train[feat].mean())
                X_val[feat] = X_val[feat].fillna(X_train[feat].mean())
    return X_train, X_val

def frequency_encoding(train_df, val_df, column):
    """
    Apply frequency encoding on the training set and use the same encoding to impute the validation set.
    
    Parameters:
    train_df (pd.DataFrame): Training dataset.
    val_df (pd.DataFrame): Validation dataset.
    column (str): Column to encode.
    
    Returns:
    train_encoded (pd.DataFrame): Encoded training set.
    val_encoded (pd.DataFrame): Encoded validation set.
    freq_map (dict): Mapping of frequency counts for the column.
    """
    # Compute frequency encoding for the training set
    freq_map = train_df[column].value_counts(normalize=True)  # Relative frequency
    train_df[f"{column}_freq"] = train_df[column].map(freq_map)

    # Impute frequency encoding on the validation set using the same mapping
    val_df[f"{column}_freq"] = val_df[column].map(freq_map)

    # Handle unseen categories in validation by imputing 0 frequency
    val_df[f"{column}_freq"] = val_df[f"{column}_freq"].fillna(0)
    
    train_df = train_df.drop(columns=[column])
    val_df = val_df.drop(columns=[column])

    # Return encoded datasets and frequency map
    return train_df, val_df, freq_map

def target_guided_ordinal_encoding(X_train, X_val, categorical_column, target_column, y_train, i):
    # Combine X_train with y_train temporarily to calculate means
    X_train_encoded = X_train.copy()
    X_val_encoded = X_val.copy()
    X_train_encoded[target_column] = y_train

    means = X_train_encoded.groupby(categorical_column)[target_column].mean()
    #print(means)

    sorted_means = means.sort_values(by=target_column)
    #print(sorted_means)
    # if i == 1:
    #     print(f"Showing sorted means for {categorical_column}")
    #     lst_names = sorted_means.index.tolist()
    #     lst_values = sorted_means.values.tolist()
    #     dict_final = dict(zip(lst_names, lst_values))
    #     print(dict_final)
    
    ordinal_mapping = {category: rank for rank, category in enumerate(sorted_means.index, start=1)}
    # if i == 1:
    #     print(f"Showing ordinal mapping for {categorical_column}")
    #     print(ordinal_mapping)
    #     print("--------------------------------")
        
    X_train_encoded[f"{categorical_column}_encoded"] = X_train_encoded[categorical_column].map(ordinal_mapping)
    X_val_encoded[f"{categorical_column}_encoded"] = X_val_encoded[categorical_column].map(ordinal_mapping)

    #X_train_encoded = X_train_encoded.drop(columns=[categorical_column])
    X_train_encoded = X_train_encoded.drop(columns=[target_column[0]])
    #X_val_encoded = X_val_encoded.drop(columns=[categorical_column])
    X_train_encoded = X_train_encoded.fillna(1)
    X_val_encoded = X_val_encoded.fillna(1)

    return X_train_encoded, X_val_encoded, ordinal_mapping

# ----------------------------------- Model Optimization ----------------------------------- # 


selected_features=['Attorney/Representative',
 'IME-4 Count',
 'Accident Date_year',
 'Accident Date_assembly_gap_days',
 'C3-C2_gap_days',
 'C2_missing',
 'C3_missing',
 'C3_Accident_gap_weeks',
 'Hearing_C3 gap_months',
 'Hearing_C2 gap_months',
 'Days to Assembly',
 'Days to First Hearing',
 'Average Weekly Wage_log',
 'Carrier Name_encoded',
 'Carrier Type_encoded',
 'Industry Code Description_encoded',
 'WCIO Cause of Injury Description_encoded',
 'WCIO Nature of Injury Description_encoded',
 'WCIO Part Of Body Description_encoded',
 'Carrier Name_freq',
 'Carrier Type_freq',
 'Industry Code Description_freq',
 'WCIO Nature of Injury Description_freq',
 'WCIO Part Of Body Description_freq']

naive_features = [feat.replace("_encoded", "") for feat in selected_features]
naive_features = [feat.replace(f"_freq", "") for feat in naive_features]
naive_features = set(naive_features)
naive_features = list(naive_features)

cat_feats = [feat for feat in naive_features if feat in cat_feats]

log_reg_params = {'C': [1], "solver":["lbfgs"], "class_weight":[None, "balanced"]}
nb_params = {"var_smoothing": [1e-9, 0.1]}
#knn_params = {'weights' : ['uniform','distance'], "n_neighbors":[5, 7]}
rfc_params = {"max_depth": [6], "class_weight": ["balanced"]}
#gb_params = {"max_depth": [3, 9], "n_estimators": [100, 300], "learning_rate": [0.1, 0.01]}
#xgboost_params = {"max_depth": [6, 9], "learning_rate": [0.3, 0.03]}
#hbg_params = {"max_depth": [6, 9], "learning_rate": [0.1, 0.01], "max_iter":[100, 200], "class_weight":[None, "balanced"]}
catboost_params = {'iterations': [1000], 'depth':[6], 'boosting_type': ['Ordered'], "auto_class_weights": ["SqrtBalanced"], "loss_function": ["MultiClassOneVsAll"]} #"l2_leaf_reg":[4] # }
nn_params = {'hidden_layer_sizes': [(25, 8)], "learning_rate_init": [0.01]}
#svc_params = {"C": [1, 0.1], "class_weight": [None, "balanced"]}


models = [
    ('Logistic Regression', LogisticRegression(random_state=42), log_reg_params),
    ("NB", GaussianNB(), nb_params), # ele ajusta automaticamente the class priors as a parameter
    #("KNNC", KNeighborsClassifier(), knn_params),
    ("Random Forest", RandomForestClassifier(random_state=42), rfc_params),
    #("Gradient Boosting", GradientBoostingClassifier(random_state=42), gb_params),
    #("HGB", HistGradientBoostingClassifier(random_state=42), hbg_params),
    #("xgboost", XGBClassifier(random_state=42, verbose=0), xgboost_params),
    ('CatBoost', CatBoostClassifier(random_state=42, verbose=100), catboost_params),
    ("Neural Network", MLPClassifier(random_state=42, verbose=10), nn_params),
    #("SVC", SVC(random_state=42), svc_params),
]


def optimize_thresholds(y_true, probabilities):
    best_thresholds = []
    for i in range(probabilities.shape[1]):  # Loop over each class
        precision, recall, thresholds = precision_recall_curve((y_true == i).astype(int), probabilities[:, i])
        f1_scores = 2 * (precision * recall) / (precision + recall + 1e-9)
        best_thresholds.append(thresholds[np.argmax(f1_scores)])  # Store best threshold
    return best_thresholds

def predict_with_thresholds(probabilities, thresholds):
    weighted_probs = probabilities / np.array(thresholds)  
    predictions = np.argmax(weighted_probs, axis=1)  
    return predictions


def model_predictions_global(X_train, y_train, X_val, y_val, model, clf_name, specific_target, cv_i, params):
    
    train_proba = model.predict_proba(X_train)
    val_proba = model.predict_proba(X_val)
    
    best_thresholds = optimize_thresholds(y_val, val_proba)
    
    train_predictions = predict_with_thresholds(train_proba, best_thresholds)
    val_predictions = predict_with_thresholds(val_proba, best_thresholds)
    
    
    f1_score_train = f1_score(y_train, train_predictions, average="macro")
    f1_score_val = f1_score(y_val, val_predictions, average="macro")
    
    print(f"{clf_name}...")
    print(f"Params: {params}")
    print(f"Train F1-score: {round(f1_score_train, 3)}")
    print(f"Thresholds: {best_thresholds}")
    print(f"Validation F1-score: {round(f1_score_val, 3)}")
    print(classification_report(y_val, val_predictions))
    
    return params, f1_score_train, f1_score_val 

n_splits = 3
stratified_kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

X = df[naive_features]
y = df[ordinal_target]

X.columns

y.value_counts(normalize=True)
models_params = []
for i, (train_index, val_index) in enumerate(stratified_kf.split(X, y), start=1):
    print(f"Starting CV_{i}/3 for {ordinal_target}...")
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    #--------------- Target Ordinal Encoding 
        

    print(f"Ordinal encoding...")
    X_train_encoded = X_train.copy()
    X_val_encoded = X_val.copy()
    for cat in cat_feats:
        X_train_encoded, X_val_encoded, ordinal_mapping = target_guided_ordinal_encoding(X_train_encoded, X_val_encoded, cat, ordinal_target, y_train, 1)

    
    columns = X_train_encoded.columns

    # --------------- Target Encoding
    
    # num_feats = [feat for feat in X_train.columns if X_train[feat].dtype != "object"]
    # cat_feats = [feat for feat in X_train.columns if X_train[feat].dtype == "object" and X_train[feat].nunique()]
    
    # te = TargetEncoder(target_type="multiclass", smooth=0, random_state=42)
    # X_train_encoded,  = te.fit_transform(X_train[cat_feats], y_train), 
    # X_val_encoded = te.transform(X_val[cat_feats])
    # expanded_features = []
    # for feature in te.feature_names_in_:
    #     for i in range(1, 9):  
    #         expanded_features.append(f"{feature}_x_target_{i}")

    # print(expanded_features)
    # print(f"Total Features names   {len(expanded_features)} == transformed columns shape {X_train_encoded.shape[1]}")
    # X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=expanded_features, index=X_train.index)
    # X_val_encoded_df = pd.DataFrame(X_val_encoded, columns=expanded_features, index=X_val.index)
    
    # X_train_encoded = pd.concat([X_train[num_feats], X_train_encoded_df], axis=1)
    # X_val_encoded = pd.concat([X_val[num_feats], X_val_encoded_df], axis=1)
    
    # print(f"Final shape of the all features: {X_train_encoded.shape[1]}")
    
    # --------------- Frequency Encoding
    print(f"Frequency encoding...")
    for cat in cat_feats:
        X_train_encoded, X_val_encoded, freq_map = frequency_encoding(X_train_encoded, X_val_encoded, cat)

    
    X_train_encoded  = X_train_encoded[selected_features]
    X_val_encoded = X_val_encoded[selected_features]
    
    #print(f"X_train columns: {X_train_encoded.columns}")
    
    # --------------- Imputing missing values
    print(f"Impuiting missing values...")
    X_train_imputed, X_val_imputed = num_imputing(X_train_encoded, X_val_encoded)
        
    for clf_name, clf, param_grid in models:
        # -------------- Scaling and Normalizing for some models
        if clf_name in ["Logistic Regression", "KNNC", "Neural Network", "NB", "SVC"]:
            X_train_imputed, X_val_imputed = num_scaling(X_train_imputed, X_val_imputed)
        

        # -------------- Get the possible combinations of hyperparameters for each model
        print(f"Testing combinations for {clf_name}...")
        scores_dict = {}

        if param_grid:
            keys, values = zip(*param_grid.items())
            for combination in itertools.product(*values):
                params = dict(zip(keys, combination))
                if params:
                    if clf_name == "CatBoost":
                        clf = CatBoostClassifier(**params, verbose=10)
                    else:
                        clf.set_params(**params)
                    clf.fit(X_train_imputed, y_train)
                    params, train_score, val_score = model_predictions_global(X_train_imputed, y_train, X_val_imputed, y_val, clf, clf_name, ordinal_target, i, params)
                    scores_dict[(clf_name, tuple(params.items()))] = val_score
                models_params.append({
                    "clf_name": clf_name,
                    "CV":f"CV_{i}",
                    "params": params,
                    "train_score": train_score,
                    "validation_score": val_score})
                # ----------- OvRClassifier for the model
                if clf_name == "Random Forest":
                    clf.set_params(**params)
                    clf_ovr = OneVsRestClassifier(clf)
                    clf_ovr.fit(X_train_imputed, y_train)
                    params, train_score, val_score = model_predictions_global(X_train_imputed, y_train, X_val_imputed, y_val, clf_ovr, f"OVR_{clf_name}", ordinal_target, i, params)
                    scores_dict[(f"OVR_{clf_name}", tuple(params.items()))] = val_score
                    models_params.append({
                        "clf_name": f"OVR_{clf_name}",
                        "CV":f"CV_{i}",
                        "params": params,
                        "train_score": train_score,
                        "validation_score": val_score})
        best_params, best_score = max(scores_dict.items(), key=lambda x: x[1])
        print(f"Best params for {clf_name}:")
        
        print("--------------------------------------------------")

    print("--------------------------------------------------")
    
    
    
    
    
    
    
# ---------------- OVR voting classifier
    
models_params

In [5]:
models_params

[{'clf_name': 'Logistic Regression',
  'CV': 'CV_1',
  'params': {'C': 1, 'solver': 'lbfgs', 'class_weight': None},
  'train_score': 0.42633979167407554,
  'validation_score': 0.41935807691859484},
 {'clf_name': 'Logistic Regression',
  'CV': 'CV_1',
  'params': {'C': 1, 'solver': 'lbfgs', 'class_weight': 'balanced'},
  'train_score': 0.3799675746393409,
  'validation_score': 0.3775670926529801},
 {'clf_name': 'NB',
  'CV': 'CV_1',
  'params': {'var_smoothing': 1e-09},
  'train_score': 0.30907462369581584,
  'validation_score': 0.3102782397360566},
 {'clf_name': 'NB',
  'CV': 'CV_1',
  'params': {'var_smoothing': 0.1},
  'train_score': 0.351028190981727,
  'validation_score': 0.3505837214917073},
 {'clf_name': 'Random Forest',
  'CV': 'CV_1',
  'params': {'max_depth': 6, 'class_weight': 'balanced'},
  'train_score': 0.3687771854026147,
  'validation_score': 0.3499940558725009},
 {'clf_name': 'OVR_Random Forest',
  'CV': 'CV_1',
  'params': {'max_depth': 6, 'class_weight': 'balanced'},


| Model               | Parameters                                                                                     | Train Score | Validation Score |
|---------------------|-----------------------------------------------------------------------------------------------|-------------|-------------------|
| Logistic Regression | {'C': 1, 'solver': 'lbfgs', 'class_weight': Balanced''}                                             | 0.426339    | 0.419358          |
| NB                  | {'var_smoothing': 0.1}                                                                        | 0.351028    | 0.350583          |
| Random Forest       | {'max_depth': 6, 'class_weight': 'balanced'}                                                  | 0.3687      | 0.349994          |
| OVR_Random Forest   | {'max_depth': 6, 'class_weight': 'balanced'}                                                  | 0.332545    | 0.313165          |
| CatBoost            | {'iterations': 1000, 'depth': 6, 'boosting_type': 'Ordered', 'auto_class_weights': 'SqrtBalanced','loss_function': 'MultiClassOneVsAll'} | 0.560474    | 0.485403     |
| Neural Network      | {'hidden_layer_sizes': (25, 8), 'learning_rate_init': 0.01}                                   | 0.45147    |       0.446937     |